## Importing Packages

In [1]:
import xarray as xr
import pandas as pd
import numpy  as np
import scipy

import netCDF4
from netCDF4 import Dataset

import cartopy.crs as ccrs
import cartopy
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib

import sys
import subprocess

import seaborn as sns

import datetime

import glob
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib
%matplotlib inline

Using matplotlib backend: Qt5Agg


## Dataset

#### Map Information

In [3]:
shp = 'd:/00_Masters/01_Dados/Shapes/brasil_UF.shp'

extent = [-39.1,-33, -14.3,-4.3] # lonmin lonmax latmin latmax
proj = ccrs.PlateCarree()

#### Glorys Dataset

In [4]:
# # Get a list of all .nc files available in different folders
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
dsmerged = xr.open_mfdataset(filenames)

# # Corrige erro de datas duplicadas pelo mfdataset
_, index = np.unique(dsmerged['time'], return_index=True)
dsmerged = dsmerged.isel(time=index)

             Defining DEPTHS of interest 

In [5]:
Depth_0 = dsmerged.sel(depth=0,method='nearest')
Depth_0['speed'] = np.sqrt(Depth_0['uo']**2+Depth_0['vo']**2)
Depth_0['direction'] = np.mod(180+np.rad2deg(np.arctan2(Depth_0['uo'],Depth_0['vo'])),360)
# Depth_0['direction'][Depth_0['direction']<0] += 360
vN_sup = Depth_0['vo'] #vetor perpendicular importante em superfície


Depth_30 = dsmerged.sel(depth=30,method='nearest')
Depth_30['speed'] = np.sqrt(Depth_30['uo']**2+Depth_30['vo']**2)
Depth_30['direction'] = np.mod(180+np.rad2deg(np.arctan2(Depth_30['uo'],Depth_30['vo'])),360)
# Depth_30['direction'][Depth_30['direction']<0] += 360
uE_30m = Depth_30['uo'] #vetor transversal importante na camada de "fundo"

depths = [0,30]

# Climatology

### Monthly Climatology

In [6]:
Month_climatology_0 = Depth_0.groupby("time.month").mean()
Month_climatology_30 = Depth_30.groupby("time.month").mean()

M_Cs = [Month_climatology_0,Month_climatology_30]

Months = [1,2,3,4,5,6,7,8,9,10,11,12]

##### File - Max & Min

In [ ]:
Months_names = ['January','February','March',
                'April','May','June',
                'July','August','September',
                'October','November','December']

for n,d in enumerate(depths):
    f = open("D:/00_Masters/02_Resultados_Parciais/Climatologia/Ocean_Curr/Outputs/Limite_Mensal_Curr_uEvN_d{}.txt".format(d), "w+")
    for i,m in enumerate(Months):
        print('Processing: ' + Months_names[i])
        cc = M_Cs[n].sel(month=m)
        vmin_uE = cc['uo'].min().values
        vmax_uE = cc['uo'].max().values
        vmin_vN = cc['vo'].min().values
        vmax_vN = cc['vo'].max().values      
        print('\n' + Months_names[i] + ':', file=f)
        print('uE - Min.Temp =', vmin_uE, file=f)
        print('uE - Max.Temp =', vmax_uE, file=f)
        print('vN - Min.Temp =', vmin_vN, file=f)
        print('vN - Max.Temp =', vmax_vN, file=f)
        print('Done: '+ Months_names[i])
    
    f.close()  
    print('\nFile Closed')
    if n < np.size(depths)-1:
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')

##### Figure - All maps in ONE

In [17]:
Versao = 0

Months_names = ['JAN','FEB','MAR',
                'APR','MAY','JUN',
                'JUL','AUG','SEP',
                'OCT','NOV','DEC']
                
## IMPORTANTE FIXAR VALORES PARA OS MINIMOS E MAXIMOS ##
Tmin = [-1,-1]
Tmax = [1,1]
Smin = 0
Smax = .2
ArrowC = sns.color_palette("coolwarm")
step = 5
depths= [0,30]

for i,d in enumerate(depths):

    fig = plt.figure(figsize=(16.53,11.69), dpi = 300, tight_layout=True) # open a new figure window and set the resolution
    fig.suptitle('Climatology\nDepth = '+str(round(float(M_Cs[i]['depth'].values),2))+'m', fontsize=16, y=0.92)
    fig.subplots_adjust(left=None, bottom=0.1, right=0.835, top=.7, wspace=0.1, hspace=0.005)
    fig.patch.set_facecolor('white')
    
    for n,m in enumerate(Months):
        if n>0:
            print('Done: '+ Months_names[n-1])

        print('Processing: ' + Months_names[n])

        cc = M_Cs[i].sel(month=m).squeeze()
        del cc['month']
        del cc['depth']
        uE = cc['uo']
        vN = cc['vo']
        # speed = cc['speed']

        ax = plt.subplot(2, 6, n+1, projection=proj)
        ax.set_extent(extent)
        ax.text(0.9, 0.02, Months_names[n], fontweight='bold', fontsize=12, transform=ax.transAxes, ha="right",)

        plt.rcParams.update({'font.size': 10})

        shape_feature = ShapelyFeature(Reader(shp).geometries(),
                                        ccrs.PlateCarree(), edgecolor='gray',linewidth=.5)
        ax.add_feature(shape_feature,facecolor='oldlace')

        im = cc['speed'].plot.contourf(ax=ax, transform=proj, vmin=Smin, vmax=Smax, add_colorbar=False, alpha=0.5)
        lon, lat = np.meshgrid(cc['longitude'],cc['latitude'])
        ax.quiver(lon[::step,::step],lat[::step,::step],cc['uo'][::step,::step],cc['vo'][::step,::step],
                    pivot='middle',scale=2,width=.008,headwidth=2,headlength=2)#,scale=0.55,)#,color=ArrowC)

        gl = ax.gridlines(crs=proj, draw_labels=True,
                          linewidth=1, color='gray', alpha=0.5, linestyle='--')
        gl.xlocator = mticker.FixedLocator(np.arange(-34,-39.5,-2))
        gl.ylocator = mticker.FixedLocator(np.arange(-5,-14.5,-2.5)) 
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER

        if n != 0 and n != 6:
            gl.ylabels_left = False

        if n < 6:
            gl.xlabels_bottom = False

    cbar_ax = fig.add_axes([1.05, 0.08, 0.01, 0.8])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation="vertical") #shrink=.8
    cbar.set_label(label='Speed (m/s)', size=12)
    cbar.ax.tick_params(labelsize=12)

    print('Done: '+ Months_names[n])

    plt.savefig('D:/00_Masters/02_Resultados_Parciais/Climatologia/Ocean_Curr/Figuras/Curr-Clim_Mensal_d'+str(d)+'_v'+ str(Versao) + '.png',
                bbox_inches='tight') # Save image as PNG file without large white borders

    print('Picture: Saved')
    plt.close()
    if i < np.size(depths)-1:
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')
    plt.show()

Processing: JAN
Done: JAN
Processing: FEB
Done: FEB
Processing: MAR
Done: MAR
Processing: APR
Done: APR
Processing: MAY
Done: MAY
Processing: JUN
Done: JUN
Processing: JUL
Done: JUL
Processing: AUG
Done: AUG
Processing: SEP
Done: SEP
Processing: OCT
Done: OCT
Processing: NOV
Done: NOV
Processing: DEC
Done: DEC
Picture: Saved

Starting Next Depth

Processing: JAN
Done: JAN
Processing: FEB
Done: FEB
Processing: MAR
Done: MAR
Processing: APR
Done: APR
Processing: MAY
Done: MAY
Processing: JUN
Done: JUN
Processing: JUL
Done: JUL
Processing: AUG
Done: AUG
Processing: SEP
Done: SEP
Processing: OCT
Done: OCT
Processing: NOV
Done: NOV
Processing: DEC
Done: DEC
Picture: Saved

Done


### Seasonal Climatology

In [20]:
Season = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]] 
Season_names = ['SUMMER','AUTUMN','WINTER','SPRING']

Season_climatology_0 = {}
Season_climatology_30 = {}
for s,sn in zip(Season,Season_names):
    Season_climatology_0[sn] = Depth_0.sel(time = np.in1d(Depth_0['time.month'], s)).mean('time')
    Season_climatology_30[sn] = Depth_30.sel(time = np.in1d(Depth_30['time.month'], s)).mean('time')

S_Cs = [Season_climatology_0,Season_climatology_30]

In [21]:
Versao = 0

## IMPORTANTE FIXAR VALORES PARA OS MINIMOS E MAXIMOS ##
Tmin = [-1,-1]
Tmax = [1,1]
Smin = 0
Smax = .2
ArrowC = sns.color_palette("coolwarm")
step = 5
depths= [0,30]


for i,d in enumerate(depths):

    fig = plt.figure(figsize=(3.4,6.4),dpi=300)
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.85, top=0.8, wspace=0.05, hspace=0.05)
    fig.suptitle('Climatology\nDepth = '+str(round(float(S_Cs[i]['SUMMER']['depth'].values),2))+'m', fontsize=12, y=0.85)
    fig.patch.set_facecolor('white')

    for n,s in enumerate(Season):
        if n > 0:
            print('Done: '+ Season_names[n-1])

        print('Processing: ' + Season_names[n])

        cS = S_Cs[i][Season_names[n]]
        del cS['depth']
        uE = cS['uo']
        vN = cS['vo']

        ax = plt.subplot(2, 2, n+1, projection=proj)
        ax.set_extent(extent)
        ax.text(0.9, 0.02, Season_names[n], fontweight='bold', fontsize=10, transform=ax.transAxes, ha="right",)

        gl = ax.gridlines(crs=proj, draw_labels=True,
                          linewidth=1, color='gray', alpha=0.5, linestyle='--')
        gl.xlocator = mticker.FixedLocator(np.arange(-34,-39.5,-2))
        gl.ylocator = mticker.FixedLocator(np.arange(-5,-14.5,-2.5)) 
        gl.xlabels_top = False
        gl.ylabels_right = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        plt.rcParams.update({'font.size': 8})

        if n != 0 and n != 2:
            gl.ylabels_left = False

        if n < 2:
            gl.xlabels_bottom = False

        shape_feature = ShapelyFeature(Reader(shp).geometries(),
                                        ccrs.PlateCarree(), edgecolor='gray',linewidth=.5)
        ax.add_feature(shape_feature,facecolor='oldlace')

        im = cS['speed'].plot.contourf(ax=ax, transform=proj, vmin=Smin, vmax=Smax, add_colorbar=False, alpha=0.5)
        lon, lat = np.meshgrid(cS['longitude'],cS['latitude'])
        ax.quiver(lon[::step,::step],lat[::step,::step],cS['uo'][::step,::step],cS['vo'][::step,::step],
                    pivot='middle',scale=2,width=.008,headwidth=2,headlength=2)#,scale=0.55,headwidth=2,headlength=2,)#,color=ArrowC)


    print('Done: '+ Season_names[n])   

    cbar_ax = fig.add_axes([0.90, 0.115, 0.02, 0.67])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation="vertical")
    cbar.set_label(label='Speed (m/s)', size=8)
    cbar.ax.tick_params(labelsize=8)

    plt.savefig('D:/00_Masters/03_Figuras_Finais/Climatologia/Ocean_Curr/Figuras/Curr-Clim_Sazonal_d'+str(d)+'_v'+ str(Versao) + '.png',
                bbox_inches='tight')

    print('Picture: Saved')
    plt.close()
    if i < np.size(depths)-1:
        print('\nStarting Next Depth\n')
    else:
        print('\nDone')
    # plt.show()

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING
Picture: Saved

Starting Next Depth

Processing: SUMMER
Done: SUMMER
Processing: AUTUMN
Done: AUTUMN
Processing: WINTER
Done: WINTER
Processing: SPRING
Done: SPRING
Picture: Saved

Done
